# Trapdoor Detection - LOMMA

This notebook conducts the trapdoor detection on LOMMA attacks.

## Prerequisites

1. Conducted the Model Training and the Model Inversion metrics to produce reconstructed samples, as instructed in [README.md](https://github.com/ntuaislab/Trap-MID/blob/main/README.md).
2. Acquired the trapdoor signature from [trapdoor_detection.ipynb](https://github.com/ntuaislab/Trap-MID/blob/main/visualization/trapdoor_detection/trapdoor_detection.ipynb).

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys
import random

import numpy as np
import torch
from torch import nn
from tqdm import tqdm

sys.path.append("<PATH_TO_TRAP-MID_REPO>") # e.g., "../.."

import utils
import classify

In [2]:
def set_random_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
file = "<PATH_TO_CONFIG_FILE>" # e.g., "../../config/celeba/classify_trap.json"
args = utils.load_json(json_file=file)
channel = args["dataset"]["channel"]
height = args["dataset"]["height"]
width = args["dataset"]["width"]
n_classes = args["dataset"]["n_classes"]

In [4]:
net = classify.VGG16(n_classes)
net = torch.nn.DataParallel(net).cuda()
ckpt_path = '<MODEL_CHECKPOIINT_PATH>'
ckp_T = torch.load(ckpt_path)
state_dict = ckp_T['state_dict']
net.load_state_dict(state_dict, strict=False)
for param in net.parameters():
    param.requires_grad = False
net.eval()

/home/jacky/.pyenv/versions/3.9.16/envs/plg/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jacky/.pyenv/versions/3.9.16/envs/plg/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DataParallel(
  (module): VGG16(
    (feature): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=

## Recovery Analysis

In [11]:
attack_method = 'kedmi' # gmi, kedmi
attack_dir = '<PATH_TO_LOMMA_ATTACK_RESULTS>'
eval_seed = 9
attack_result = np.load(
    f'{attack_dir}/{attack_method}_300ids/celeba_VGG16{"_vib" if "MID" in attack_dir else ""}/ours/latent/attack{eval_seed}_full.npy',
    allow_pickle=True
).item()
feat_recovered = None
pred_recovered = None
iden = None

set_random_seed(0)
with torch.no_grad():
    for i in tqdm(range(len(attack_result['imgs']))):
        fake = torch.from_numpy(attack_result['imgs'][i])
        fake_iden = torch.from_numpy(attack_result['label'][i])

        feats, out_prob = net(fake)
        feats = feats.cpu()
        if feat_recovered is None:
            feat_recovered = feats
            pred_recovered = out_prob.argmax(dim=1).cpu()
            iden = fake_iden
        else:
            feat_recovered = torch.vstack([feat_recovered, feats])
            pred_recovered = torch.hstack([pred_recovered, out_prob.argmax(dim=1).cpu()])
            iden = torch.hstack([iden, fake_iden])

feat_recovered.shape, iden.shape

100%|██████████| 100/100 [00:00<00:00, 111.73it/s]


(torch.Size([5000, 2048]), torch.Size([5000]))

In [ ]:
result_dir = '<PATH_TO_SIGNATURE_RESULTS>'

classwise_signature_unit = torch.load(f'{result_dir}/classwise_signature_unit.tar')

In [13]:
feat_recovered_unit = nn.functional.normalize(feat_recovered, p=2)
classwise_cos_recovered = torch.tensor([feat_recovered_unit[i] @ classwise_signature_unit[pred_recovered[i]] for i in range(feat_recovered_unit.shape[0])])
classwise_cos_recovered.mean()

tensor(0.2590)

## Results Saving

In [ ]:
torch.save(classwise_cos_recovered, f'{result_dir}/classwise_cos_lomma_{attack_method}.tar')